In [ ]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227137 sha256=83eacf316a62d2578200e04f08a0bfbc237b44c5744dc1ce54811e40be9ee3da
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import fasttext

In [ ]:
model_en = fasttext.load_model('/content/drive/MyDrive/cc.en.300.bin')

In [ ]:
model_en.get_nearest_neighbors('good')

[(0.7517593502998352, 'bad'),
 (0.7426098585128784, 'great'),
 (0.7299689054489136, 'decent'),
 (0.7123614549636841, 'nice'),
 (0.6796907186508179, 'Good'),
 (0.6737031936645508, 'excellent'),
 (0.669592022895813, 'goood'),
 (0.6602178812026978, 'ggod'),
 (0.6479219794273376, 'semi-good'),
 (0.6417751908302307, 'good.Good')]

In [ ]:
model_en

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='latin-1')

In [ ]:
df = data

In [ ]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
df = df.rename(columns={'v1':'label','v2':'Text'})
df['label_enc'] = df['label'].map({'ham':0,'spam':1})
df.head()


,label,Text,label_enc
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
df

,label,Text,label_enc
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will Ì_ b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [ ]:
df['label'] = '__label__'+ df['label']

In [ ]:
df

,label,Text,label_enc
0,__label__ham,go until jurong point crazy available only in ...,0
1,__label__ham,ok lar joking wif u oni,0
2,__label__spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,__label__ham,u dun say so early hor u c already then say,0
4,__label__ham,nah i don't think he goes to usf he lives arou...,0
...,...,...,...
5567,__label__spam,this is the 2nd time we have tried 2 contact u...,1
5568,__label__ham,will ì_ b going to esplanade fr home,0
5569,__label__ham,pity was in mood for that so any other suggest...,0
5570,__label__ham,the guy did some bitching but i acted like i'd...,0


In [ ]:
df['label'] = df['label'] + ' ' + df['Text']

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(r'[ \n]+', ' ', text)
    return text.strip().lower()

In [ ]:
df.shape

(5572, 3)

In [ ]:
import re

In [ ]:
df.label = df.label.map(preprocess)

In [ ]:
df.label

0       __label__ham go until jurong point crazy avail...
1                    __label__ham ok lar joking wif u oni
2       __label__spam free entry in 2 a wkly comp to w...
3       __label__ham u dun say so early hor u c alread...
4       __label__ham nah i don't think he goes to usf ...
                              ...                        
5567    __label__spam this is the 2nd time we have tri...
5568    __label__ham will ì_ b going to esplanade fr home
5569    __label__ham pity was in mood for that so any ...
5570    __label__ham the guy did some bitching but i a...
5571               __label__ham rofl its true to its name
Name: label, Length: 5572, dtype: object

In [ ]:
df.to_csv("messages.txt", columns=['label'], header=None, index=False)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
train.shape, test.shape

((4457, 3), (1115, 3))

In [ ]:
model = fasttext.train_unsupervised("messages.txt")

In [ ]:
train.to_csv("messages.train", columns=["label"], index=False, header=False)
test.to_csv("messages.test", columns=["label"], index=False, header=False)

In [ ]:
import fasttext

model = fasttext.train_supervised(input="messages.train")
model.test("messages.test")

(1115, 0.9865470852017937, 0.9865470852017937)

In [ ]:
model.predict("Congratulations! You've won a free vacation to an exotic island! Click here to claim your prize now! ")

(('__label__spam',), array([0.993689]))

In [ ]:
model.predict('Important notice: Your account has been suspended due to suspicious activity. Verify your identity by logging in now.')

(('__label__ham',), array([0.93352067]))

In [ ]:
model.predict("Don't miss this opportunity! Earn $1000 a day from home with our proven money-making system.")

(('__label__ham',), array([0.84951341]))

In [ ]:
model.predict("You've been selected for a special promotion! Simply reply 'YES' to this message to redeem your exclusive discount code.")

(('__label__spam',), array([0.98209906]))

In [ ]:
model.predict("Congratulations! You've won a lottery prize of $1,000,000. To claim your winnings, reply with your name, address, and social security number.")

(('__label__spam',), array([0.99051964]))

In [ ]:
model.predict("Hey! Just wanted to check in and see how you're doing. Let me know if you want to catch up sometime!")

(('__label__ham',), array([0.99611235]))

In [ ]:
model.predict("Hey, I found that book you were looking for at the bookstore. Shall I pick it up for you?""")

(('__label__ham',), array([0.99650812]))

In [41]:
train

,label,Text,label_enc
2511,__label__ham er yep sure props,er yep sure props,0
2779,__label__ham well i might not come then,well i might not come then,0
5175,__label__ham raji pls do me a favour pls conve...,raji pls do me a favour pls convey my birthday...,0
157,__label__ham hello my love what are you doing ...,hello my love what are you doing did you get t...,0
2507,__label__ham yup,yup,0
...,...,...,...
1283,__label__ham yes i thought so thanks,yes i thought so thanks,0
5538,__label__ham i can't believe how attached i am...,i can't believe how attached i am to seeing yo...,0
5053,__label__spam double mins double txt 1 2 price...,double mins double txt 1 2 price linerental on...,1
3551,__label__ham lol u still feeling sick,lol u still feeling sick,0


In [ ]:
test

,label,Text,label_enc
1886,__label__spam dear 0776xxxxxxx u've been invit...,dear 0776xxxxxxx u've been invited to xchat th...,1
4082,__label__ham good afternoon my love it was goo...,good afternoon my love it was good to see your...,0
936,__label__ham since when which side any fever a...,since when which side any fever any vomitin,0
1757,__label__ham miss ya need ya want ya love ya,miss ya need ya want ya love ya,0
2023,__label__ham is there any movie theatre i can ...,is there any movie theatre i can go to and wat...,0
...,...,...,...
4053,__label__ham dont make ne plans for nxt wknd c...,dont make ne plans for nxt wknd coz she wants ...,0
2689,__label__ham ya ok vikky vl c witin lt gt mins...,ya ok vikky vl c witin lt gt mins and il reply u,0
2122,__label__ham do you know when the result,do you know when the result,0
963,__label__ham yo chad which gymnastics class do...,yo chad which gymnastics class do you wanna ta...,0
